In [41]:
import json
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from ast import literal_eval
import numpy as np
from scipy import spatial

In [42]:
# enter paratmeters 
ingestion_src = './project-120722-1-ingestion.json'
ModelName = 'faster_rcnn_inception_resnet_v2'
detection_src = './project-120722-1-detections-'+ ModelName +'.csv'
processed_video_src = 'dataset/processed1.avi'
aim_score = 0.8

In [43]:
f = open(ingestion_src)
data = json.load(f)
source = data['source-video']
df = pd.read_csv(detection_src,converters={"boxes": literal_eval})
frameNo = 1 
df_currentFrame = df[df['frame']==frameNo]
def get_normal_coord(decimal_coords,im_height,im_width):
    ymin, xmin, ymax, xmax = decimal_coords
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
    return [int(left),int(right),int(top),int(bottom)]


In [44]:
cap = cv2.VideoCapture(processed_video_src)
ret, frame = cap.read()
im_height,im_width,_ = frame.shape
out =cv2.VideoWriter('demo-1-('+ str(aim_score) + ')-'+ ModelName +'.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (im_width,im_height))


In [45]:
cap.get(7)

1000.0

In [46]:
normalized = []
cap = cv2.VideoCapture(processed_video_src)
frameNo = 0 
if (cap.isOpened()== False): 
    print("Error opening video stream or file")
else:       
    while(cap.isOpened()):
        ret, frame = cap.read()
        
        if ret == True:
            pass
        else: 
            break    
        im_height,im_width,_ = frame.shape
        image_np = frame.copy()
        frameNo = frameNo + 1 
        # Get detections here 
        df_currentFrame = df[df['frame']==frameNo]
        for ind,row in df_currentFrame.iterrows():
        # DETECTIONS 
            detections = row['boxes']
            score = row['scores']
            if score> aim_score:
            # plot boxes on 
                left, right, top, bottom = get_normal_coord(detections,im_height,im_width)
                image_np = cv2.rectangle(image_np, (int(left),int(top)), (int(right),int(bottom)), (0,0,255), 2)
                centeroid =(int((int(right) + int(left))/2),( int((int(top) + int(bottom))/2)))
                normalized.append(
                    {
                        'frame': frameNo,
                        'centeroid':centeroid
                    }
                )
#         out.write(image_np)
        
# out.release()
       

In [47]:
normalized_points = pd.DataFrame(normalized)

In [48]:
# split = 7 
# currentFrameIndex = 1
# clearFlag = True
# continuityRow = []

# while True:
#     currFrame = normalized_points[normalized_points['frame']==currentFrameIndex]
#     currFrame = currFrame.reset_index()
#     nextFrame = normalized_points[normalized_points['frame']==currentFrameIndex + 1]
#     nextFrame = nextFrame.reset_index()
    
#     if clearFlag == True:
#         clearFlag = False
#         continuityRow = np.ones(len(currFrame))
#         currFrame['continuity'] = continuityRow
#     else:
#         print(continuityRow)
#         currFrame['continuity'] = continuityRow
        
#         pass
    
#     currDistAr = []
#     currIndAr = []
#     for ind,row in currFrame.iterrows():
        
#         det_centeroid = row['centeroid']
#         distance,index = spatial.KDTree(list(nextFramePoints)).query(list(det_centeroid))
#         currDistAr.append(distance)
#         currIndAr.append(index)
    
    
    
#     currFrame['distance'] =  currDistAr  
#     currFrame['index'] = currIndAr
#     currFrame['continuity'] = currFrame['distance']<currFrame['distance'].mean()
    
#     nextContinuityIndexes = currFrame[currFrame['continuity']==True]['index'].tolist()
    
    
#     nextContinuityArray = np.zeros(len(nextFrame),dtype=int)
#     nextContinuityArray[[nextContinuityIndexes]] = 1 
#     continuityRow = nextContinuityArray
#     print(f'continuityRow for {currentFrameIndex+1}: {continuityRow}')
    
#     currentFrameIndex = currentFrameIndex + 1 
    
#     if currentFrameIndex==4:
#         break



In [49]:
# currFrame

In [50]:
# nextFrame

In [51]:
# nextContinuity

In [52]:
# nextFrame

In [53]:
# nextFramePoints = nextFrame['centeroid'].values

In [54]:
# from scipy import spatial

In [55]:
# nextFramePoints

In [56]:
# index

In [57]:
# normalized_points[normalized_points['frame']==2]

In [58]:
# normalized_points[normalized_points['frame']==3]

# Getting Detections and making PLOTOP and COUNTEROP

In [59]:
PLOTOP = []
COUNTEROP = []
currentFrameIndex = 1
frameSkip = 5
while True:
    if len(normalized_points[normalized_points['frame']==currentFrameIndex+frameSkip]) == 0:
        break
    currDistAr= []
    currIndAr = []
    currFrame = normalized_points[normalized_points['frame']==currentFrameIndex]
    currFrame = currFrame.reset_index()
    nextFrame = normalized_points[normalized_points['frame']==currentFrameIndex + frameSkip]
    nextFrame = nextFrame.reset_index()
    nextFramePoints = nextFrame['centeroid'].values
    
    for ind,row in currFrame.iterrows():
        
        det_centeroid = row['centeroid']
        distance,index = spatial.KDTree(list(nextFramePoints)).query(list(det_centeroid))
        currDistAr.append(distance)
        currIndAr.append(index)
    
    currFrame['distance'] =  currDistAr  
    currFrame['index'] = currIndAr
    currFrame['continuity'] = currFrame['distance']<currFrame['distance'].mean()
    
    
    
    PLOTLINES = []
    for ind,row in currFrame[currFrame['continuity']==True].iterrows():
        startingPoint = row['centeroid']
        next_index = row['index']
        endingPoint = nextFrame.loc[next_index]['centeroid']
        PLOTLINES.append((startingPoint,endingPoint))
        COUNTEROP.append({
            'frame':currentFrameIndex,
            'startingPoint' : startingPoint,
            'endingPoint' : endingPoint
            
        })
        
    PLOTOP.append({
        'frame':currentFrameIndex,
        'PLOTLINES':PLOTLINES
    })
    currentFrameIndex = currentFrameIndex + 1 


In [60]:
COUNTEROP = pd.DataFrame(COUNTEROP)
PLOTOP = pd.DataFrame(PLOTOP)


# CounterOP Output

In [61]:
boxes = data['boxes']
def solve(tl,br,point):
    if point[0]>tl[0] and point[0]<br[0] and point[1]>tl[1] and point[1]<br[1] :
        return '1'
    else:
        return '0'

In [62]:
finalBoxAr = []

for ind,row in COUNTEROP.iterrows():
    start = row['startingPoint']
    end = row['endingPoint']
    for box in boxes:
        tl = (box[0],box[1])
        br = (box[2],box[3]) 
        finalBoxAr.append(solve(tl, br, start)+solve(tl, br, end))
  

In [71]:
# len(boxes)

In [64]:
# for i in range(len(boxes)):
#     COUNTEROP[f'box{i}op'] = np.array(finalBoxAr).reshape(-1,4)[:,i]
for i in range(len(boxes)):
    COUNTEROP[f'box{i}op'] = np.array(finalBoxAr).reshape(-1,len(boxes))[:,i]

In [72]:
# COUNTEROP

In [65]:
out =cv2.VideoWriter('testCounter.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (im_width,im_height))

cap = cv2.VideoCapture(processed_video_src)
frameNo = 0 
if (cap.isOpened()== False): 
    print("Error opening video stream or file")
else:       
    while(cap.isOpened()):
        ret, frame = cap.read()
        
        if ret == True:
            pass
        else: 
            break    
        im_height,im_width,_ = frame.shape
        image_np = frame.copy()
        frameNo = frameNo + 1 
        # Get detections here 
        
        print(frameNo)
        for i,rows in COUNTEROP[COUNTEROP['frame']==frameNo].iterrows():
            sp = rows['startingPoint']
            ep = rows['endingPoint']
            for j in range(len(boxes)):
                
                image_np = cv2.rectangle(image_np, (boxes[j][0],boxes[j][1]), (boxes[j][2],boxes[j][3]), (0,0,255), 2)
                if rows[f'box{j}op'] == '01':
                    print('01 detected')
                    image_np = cv2.line(image_np, 
                                            sp, 
                                            ep,
                                            (0,180,0),
                                            (30))
                elif rows[f'box{j}op'] == '10':
                    print('10 detected')
                    image_np = cv2.line(image_np, 
                                            sp, 
                                            ep,
                                            (0,0,180),
                                            (30))
                else:
                    image_np = cv2.line(image_np, 
                                            sp, 
                                            ep,
                                            (180,0,0),
                                            (5))

        
        out.write(image_np)
#         if frameNo==1000:
#             break
        
out.release()
       

1
2
3
4
5
10 detected
01 detected
6
7
01 detected
8
10 detected
01 detected
9
10 detected
10
10 detected
11
10 detected
12
10 detected
13
10 detected
14
15
16
01 detected
17
01 detected
18
01 detected
19
20
21
10 detected
22
10 detected
23
01 detected
24
01 detected
25
01 detected
26
01 detected
27
01 detected
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
10 detected
46
10 detected
47
10 detected
48
49
50
51
52
53
54
55
56
57
58
10 detected
59
60
61
62
01 detected
01 detected
63
01 detected
01 detected
64
01 detected
01 detected
65
01 detected
66
01 detected
67
68
10 detected
69
10 detected
70
10 detected
10 detected
71
10 detected
10 detected
72
10 detected
10 detected
73
10 detected
74
10 detected
75
76
77
10 detected
78
79
80
10 detected
81
82
01 detected
83
10 detected
84
10 detected
85
10 detected
86
10 detected
87
10 detected
88
89
90
91
92
93
94
95
96
97
01 detected
98
99
10 detected
100
01 detected
101
01 detected
01 detected
102
01 detected
103
104
105
10 detected
10 d

725
01 detected
726
727
10 detected
728
10 detected
729
10 detected
730
731
732
733
734
01 detected
735
01 detected
736
01 detected
737
01 detected
10 detected
738
739
740
10 detected
741
742
743
744
745
746
747
748
10 detected
749
750
751
01 detected
752
753
01 detected
754
755
01 detected
01 detected
756
01 detected
01 detected
757
01 detected
10 detected
758
01 detected
10 detected
759
01 detected
10 detected
760
761
762
763
10 detected
764
765
10 detected
766
767
768
769
770
771
772
773
774
775
776
01 detected
01 detected
777
778
01 detected
01 detected
01 detected
779
01 detected
780
01 detected
781
782
01 detected
783
784
785
786
787
788
01 detected
789
01 detected
790
791
10 detected
792
10 detected
793
10 detected
794
10 detected
795
796
797
798
799
800
801
802
803
804
10 detected
805
10 detected
806
10 detected
807
808
01 detected
809
01 detected
810
01 detected
01 detected
811
812
813
814
815
816
817
818
819
820
821
10 detected
822
10 detected
823
01 detected
824
01 detected


# Plotop Processing 

In [68]:
out =cv2.VideoWriter('testMotionOpticalFlow.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (im_width,im_height))

cap = cv2.VideoCapture(processed_video_src)
frameNo = 0 
if (cap.isOpened()== False): 
    print("Error opening video stream or file")
else:       
    while(cap.isOpened()):
        ret, frame = cap.read()
        
        if ret == True:
            pass
        else: 
            break    
        im_height,im_width,_ = frame.shape
        image_np = frame.copy()
        frameNo = frameNo + 1 
        # Get detections here 
        
        print(frameNo)
        try:
            for lineParam in PLOTOP[PLOTOP['frame']==frameNo]['PLOTLINES'][frameNo-1]:
                image_np = cv2.line(image_np, 
                                    lineParam[0], 
                                    lineParam[1],
                                    (0,180,0),
                                    (5))
                
        except Exception:
            print('No Lines Found')
        
        
        
        out.write(image_np)
#         if frameNo==100:
#             break
        
out.release()
       

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [67]:
# PLOTOP[PLOTOP['frame']==1334]['PLOTLINES']